In [1]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow as tf
from imutils.video import VideoStream
from imutils import face_utils
import imutils
import cv2
import time
import dlib

%matplotlib inline

C:\Users\Anup\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def compute_content_cost(a_C, a_G):
    m, n_H, n_W, n_C = a_G.get_shape().as_list()

    a_C_unrolled = tf.reshape(a_C, shape=[m, n_H, n_W, n_C])
    a_G_unrolled = tf.reshape(a_G, shape=[m, n_H, n_W, n_C])
    
    J_content = tf.reduce_sum(tf.square(a_C_unrolled - a_G_unrolled)) / (4 * n_H * n_W * n_C)
    
    return J_content

In [3]:
def gram_matrix(A):
    GA = tf.matmul(A, tf.transpose(A))    
    return GA

In [4]:
def compute_layer_style_cost(a_S, a_G):
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_S = tf.transpose(tf.reshape(a_S, [n_H*n_W, n_C]))
    a_G = tf.transpose(tf.reshape(a_G, [n_H*n_W, n_C]))

    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

    J_style_layer = (1./(4 * n_C**2 * (n_H*n_W)**2)) * tf.reduce_sum(tf.pow((GS - GG), 2))

    return J_style_layer

In [5]:
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

In [6]:
def compute_style_cost(model, STYLE_LAYERS):
    J_style = 0

    for layer_name, coeff in STYLE_LAYERS:
        out = model[layer_name]
        a_S = sess.run(out)
        a_G = out
        J_style_layer = compute_layer_style_cost(a_S, a_G)
        J_style += coeff * J_style_layer

    return J_style

In [7]:
def total_cost(J_content, J_style, alpha = 30, beta = 40):
    J = alpha * J_content + beta * J_style
    return J

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [9]:
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19-1.mat")

In [10]:
# content cost
def c1(model, con):
    #content_image = scipy.misc.imread(con)
    content = reshape_and_normalize_image(con)
    sess.run(model['input'].assign(content))
    out = model['conv4_2']
    a_C = sess.run(out)
    a_G = out
    
    return compute_content_cost(a_C, a_G), content

In [11]:
# style cost
def c2(model, sty):
    style_image = scipy.misc.imread(sty)
    style = reshape_and_normalize_image(style_image)
    sess.run(model['input'].assign(style))
    return compute_style_cost(model, STYLE_LAYERS)

In [12]:
def model_nn(sess, model, style):
    vs = VideoStream(src=0).start()
    time.sleep(2.0)
    
    while True:
        frame = vs.read()
        frame = imutils.resize(frame, width=400, height=300)
    
        J_content, f1 = c1(model, frame)
        J_style = c2(model, style)

        J = total_cost(J_content, J_style, alpha = 10, beta = 40)

        optimizer = tf.train.AdamOptimizer(2.0)
        train_step = optimizer.minimize(J)    

        sess.run(tf.global_variables_initializer())

        sess.run(model['input'].assign(f1))

            
        # Run the session on the train_step to minimize the total cost
        sess.run(train_step)

        # Compute the generated image by running the session on the current model['input']
        generated_image = sess.run(model['input'])

        Jt, Jc, Js = sess.run([J, J_content, J_style])
        #print("Iteration " + str(i) + " :")
        print("total cost = " + str(Jt))
        print("content cost = " + str(Jc))
        print("style cost = " + str(Js))

        cv2.imshow("Frame", generated_image)
            
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            break
            
    cv2.destroyAllWindows()
    vs.stop()

In [13]:
model_nn(sess, model, "images/style1.jpg")

C:\Users\Anup\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()


total cost = 19136707000.0
content cost = 1362.1641
style cost = 478417300.0


error: OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\core\src\array.cpp:2492: error: (-206:Bad flag (parameter or structure field)) Unrecognized or unsupported array type in function 'cvGetMat'
